In [14]:
import os
import base64
import json
import re
import requests
from dotenv import load_dotenv
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFLoader
from pdf2image import convert_from_path
import openai

# Cargar las variables de entorno desde el archivo .env
load_dotenv(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\.env')

# Obtener la clave de API desde las variables de entorno
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key  # Configurar la clave de API para OpenAI
client = openai.Client()


def load_json_from_file(file_path):
    """Carga un archivo JSON desde la ruta especificada y devuelve el contenido."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def process_json_with_openai(directory):
    """Carga y procesa todos los archivos JSON en un directorio usando la API de OpenAI."""
    
    combined_json_data = []  # Lista para almacenar el contenido combinado

    # Recorrer todos los archivos en el directorio
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            try:
                json_data = load_json_from_file(file_path)  # Cargar el contenido del JSON
                combined_json_data.append(json_data)  # Agregar a la lista
            except json.JSONDecodeError as e:
                print(f"Error al leer el archivo {filename}: {e}")
    
    # Convertir la lista combinada a un formato JSON
    combined_json_str = json.dumps(combined_json_data, indent=2)  # Indentar para mayor legibilidad

    # Crea el contenido del mensaje incluyendo el JSON
    messages = [{
        "role": "user",
        "content": (           
            "Tengo el siguiente JSON data que representa un reporte Power BI:\n\n{json_str}\n\n"
            f"{combined_json_str}\n"
            "Genera un breve informe analítico a partir de los siguientes JSON, que representa un reporte de cuadros de mando de Power BI."
            "El informe debe ser corto y debe ser útil para ayudar al usuario a detectar detalles y extraer conclusiones relevantes rápidamente."
        )
    }]
    
    client = openai.OpenAI()

    try:
        # Llamar a la API
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=15000,
            temperature=0.5,
        )

        response_content = completion.choices[0].message.content.strip()
        return response_content
    
    except Exception as e:
        print(f"Error al llamar a la API de OpenAI: {e}")
        return None



In [15]:
# Procesar el JSON con la función que has definido previamente
response = process_json_with_openai(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_API\Data_Structuring\src\images')

# Definir el nombre del informe como una cadena
nombre = "informe_completo"  # Asegúrate de que esto sea una cadena

# Guardar el contenido de la página en un archivo de texto
output_file_path = os.path.join(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_API\Data_Structuring\src\texts', f'{nombre}.txt')

# Escribir el contenido en el archivo
with open(output_file_path, 'w', encoding='utf-8') as f:
    f.write(response)  # Escribir el contenido de la respuesta en el archivo


Error al leer el archivo Eagle_Ventas_page_10_result.json: Expecting property name enclosed in double quotes: line 1 column 735 (char 734)
